In [12]:
import time
from fpylll import *
from fpylll.algorithms.bkz2 import BKZReduction as BKZ2
from sage.modules.free_module_integer import IntegerLattice
from sage import *

def CenteredMod(a, q):
    a = a.mod(q)
    if a <= floor(q/2):
        return a
    else:
        return a-q

PATH = "SET PATH OF DATA HERE"
    
# parameters
n = 2^8
q = 2^10
# # precomputed data; this is for reproducibility 
# print("Loading secret key...")
# S = load(PATH+"/sk.sobj") # secret key
# S_ = load(PATH+"/skinv.sobj") # inverse mod q

# print ("Loading gadget...")
# G = load(PATH+"/gadget.sobj") # gadget matrix

# If you want to test on your own data, uncomment below
Zq = ZZ.quo(q)
print("build secret key...")
while True:
    print("Try...")
    S0 = Matrix(Zq, [ [ randint(0,1) for _ in range(n) ] for _ in range(n) ])
    if S0.is_invertible():
        S = S0.change_ring(ZZ)
        break
S_ = S0.inverse().change_ring(ZZ) #S^-1 mod q seen in ZZ
# print("priv key is",S0)
# gadget matrix
G = block_matrix( [ [ 2^i*identity_matrix(n) for i in range(log(q,2)) ] ] )

### BUILD PUBLIC DATA ###
print("Computing cipher of Id...")
TIME = time.time()
E_Id = random_matrix(ZZ, n, log(q,2)*n, x=0, y=2) # E is secret encryption noise
C_Id = S_ * (E_Id[:n,:n] - G[:n, :n]) 
C_Id = Matrix(ZZ, [ [ CenteredMod(e, q) for e in C_Id[i] ] for i in range(n) ] ) #first block of cipher
print("Done. time: ", round(time.time() - TIME)) # takes ~20s

print("Building basis of NTRU lattice...")
TIME = time.time()
B = block_matrix([ [ q*identity_matrix(n), zero_matrix(n) ], [ C_Id, identity_matrix(n) ] ] )
B = IntegerMatrix.from_matrix(B) # type conversion, between 3-5 mins
print("Done. time: ", round(time.time() - TIME))

### PREPROC PHASE FOR FPYLLL ###
k = 100
print("Building submatrix of size 2*k =", 2*k, "...")
TIME = time.time()
Bk = B.submatrix(range(n-k,n+k), range(n-k,n+k)) # ~30s
FPLLL.set_precision(180) 
GSO_Bk = GSO.Mat(Bk, float_type='mpfr')
print("Preprocessing...")
Bk_BKZ = BKZ2(GSO_Bk) 
# f,g=GSO_Bk.LLL()[0]
print('look')
print(f,'\n',g)
print("Done. time: ", round(time.time() - TIME))   

### BKZ PHASE ###
flags = BKZ.AUTO_ABORT|BKZ.MAX_LOOPS|BKZ.GH_BND|BKZ.VERBOSE
beta = 2
par = BKZ.Param(block_size=beta, strategies=BKZ.DEFAULT_STRATEGY, flags=flags, max_loops=4)
print("BKZ reduction with beta =", beta, "...")
TIME = time.time()
DONE = Bk_BKZ(par) #actual BKZ algorithm; updates Bk in place; ~15 hours
# if it fails because infinite loop in babai, set higher precision in FPLLL.set_precision() and rerun without restarting.
print("Done. time: ", round(time.time() - TIME))
print(Bk)
### WE WON ###
# print( all(Bk[i].norm() < 2**24 for i in range(k)) )

build secret key...
Try...
Try...
Computing cipher of Id...
Done. time:  0.0
Building basis of NTRU lattice...
Done. time:  0.0
Building submatrix of size 2*k = 200 ...
Preprocessing...
look
140165891500972861127767415311334757297477816480570735901532575717579672765221212171489392644327018202900088035383969034846193847149851961591661082906530495738734185460636601059708032788268225305120040067251226533612047405603785333261269570827395533567542600084996230099530440207162530350599289132101020447759 
 1493239469420336600577802657161324795848374750185055876743560582622232360193779109171117333425338766341860983348309692998243289015442909620365263900067613009409717218816514258676437322125744209790095123709189541828478158286030084857
Done. time:  0.0
BKZ reduction with beta = 2 ...
Done. time:  34.0
[ 1024    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

In [ ]:
from sage import *
h = 7257231567493321493497732423756001924698993879741072696808433246581786362611889417289029671240997843541696187487722285762633068287623369923457879458577466240950224087015909821079480431797917376609839097610864517760515986973340148901898582075413756737310797402537478388864632901178463429574227279668004092519322204990617969134092793157225082977967228366838748193084097651575835471869030934527383379794480007872562067799484905159381690179011192017159985260435844246766711550315143517066359521598424992244464723490166447105679062078049379153154659732304112563255058750656946353654402824529058734270363154894216317570784
p = 23969137365202547728693945383611572667294904799854243194734466236017441545927679469239814785947383727854265554138290421827510545078908517696536495567625593439996528098119344504866817224169113920532528233185011693829122447604993468817512696036673804626830507903206709121383065701222707251053362179946170981868061834734684494881504724254812067180384269711822738708203454131838741703416329765575995359232573740932069147491776326045743679105041246906081872936901848272288949389026129761726749334006319072981386763830897454245553866145620689939497868469730297795063648030738668273210516497399954626983672357236110363894081
c = 6388077150013017095358415295704360631706672647932184267739118115740221804173068089559645506533240372483689997499821300861865955720630884024099415936433339512125910973936154713306915269365877588850574948033131161679256849814325373882706559635563285860782658950169507940368219930971600522754831612134153314448445006958300840618894359885321144158064446236744722180819829183828290798747455324761671198097712539900569386477647697973195787663298318786718012522378981137877863153067057280649127202971806609339007027052518049995341356359016898069863799529357397514218249272201695539181908803360181347114492616706419618151757

# Construct lattice.
v1 = vector(ZZ, [1, h])
v2 = vector(ZZ, [0, p])
m = Matrix([v1,v2])

# Solve SVP.
shortest_vector = m.LLL()[0]
f, g = shortest_vector
print(m[0],'\n',m[1])
print(f,'\n', g)

# Decrypt.
a = f*c % p % g
m = a * inverse_mod(f, g) % g
print(hex(m).decode('hex'))

(1, 7257231567493321493497732423756001924698993879741072696808433246581786362611889417289029671240997843541696187487722285762633068287623369923457879458577466240950224087015909821079480431797917376609839097610864517760515986973340148901898582075413756737310797402537478388864632901178463429574227279668004092519322204990617969134092793157225082977967228366838748193084097651575835471869030934527383379794480007872562067799484905159381690179011192017159985260435844246766711550315143517066359521598424992244464723490166447105679062078049379153154659732304112563255058750656946353654402824529058734270363154894216317570784) 
 (0, 239691373652025477286939453836115726672949047998542431947344662360174415459276794692398147859473837278542655541382904218275105450789085176965364955676255934399965280981193445048668172241691139205325282331850116938291224476049934688175126960366738046268305079032067091213830657012227072510533621799461709818680618347346844948815047242548120671803842697118227387082034541318

AttributeError: 'str' object has no attribute 'decode'